# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator #for pre-processing images

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#apply transformations on images of training set to avoid overfitting (data augmentation)
#simple geometrical transforms, zooms, rotations are used
#tool from keras API used
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#creates an instance of the ImageDataGenerator class
#rescale is feature scaling - normalize values to 0-1 range
#we use shearing, zoom, horizontal flip as augmentations
training_set = train_datagen.flow_from_directory('dataset/training_set',  #path to training set from root directory
                                                 target_size = (64, 64),  #final size of images fed into CNN
                                                 batch_size = 32,         #number of images per batch
                                                 class_mode = 'binary')   #can be binary or categorical, here we need binary outcome
#we resize images for faster performance without compromising accuracy
#flow_from_directory method connects image augmentation tool to training set images


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
#NO data augmentation is done to test set - kept intact - NO information leakage
#feature scaling is used here
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')
#import test set images here
#need to be in the same format as the training set

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()
#initialize the CNN as a sequence of layers

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
#adding a convolutional layer via the Conv2D class - 3 parameters (number of filters, kernel size which is the same as the size of the feature detector, activation function - ReLU used, input shape - eg: colored images with 64*64 will have 64*64*3 )

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#adding a pooling layer - parameters: pool size(size of pooling matrix, recommended=2), stride value(recommended=2), padding(valid(default):ignore excess cells outside matrix, extra:add fake pixels with 0 value)

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu'))
#input shape only used for the first convolutional layer

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#same as ANN

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#1 neuron only for binary classification
#signoid for binary classification and softmax for multi-class classification

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
#we use stochastic gradient descent (adam optimizer)
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
#fit method for training
#we use training_set,test_set that we made earlier in the code
#25 epochs we find perfect converging of accuracy
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 45s 179ms/step - loss: 0.6618 - accuracy: 0.5961 - val_loss: 0.6125 - val_accuracy: 0.6580
Epoch 2/25
250/250 [==============================] - 42s 167ms/step - loss: 0.5885 - accuracy: 0.6785 - val_loss: 0.5760 - val_accuracy: 0.7015
Epoch 3/25
250/250 [==============================] - 44s 174ms/step - loss: 0.5542 - accuracy: 0.7163 - val_loss: 0.5347 - val_accuracy: 0.7395
Epoch 4/25
250/250 [==============================] - 38s 152ms/step - loss: 0.5177 - accuracy: 0.7462 - val_loss: 0.5894 - val_accuracy: 0.7060
Epoch 5/25
250/250 [==============================] - 39s 157ms/step - loss: 0.5005 - accuracy: 0.7484 - val_loss: 0.4836 - val_accuracy: 0.7655
Epoch 6/25
250/250 [==============================] - 36s 146ms/step - loss: 0.4829 - accuracy: 0.7657 - val_loss: 0.5096 - val_accuracy: 0.7560
Epoch 7/25
250/250 [==============================] - 37s 146ms/step - loss: 0.4741 - accuracy: 0.7741 - val_loss: 0.4864 - val_ac

## Part 4 - Making a single prediction

In [33]:
#final test, testing finetuned model with a single picture of each class
#we use the load_img function (path)
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
test_image = load_img ('dataset/single_prediction/cat_or_dog_2.jpg', target_size= (64,64)) #with extension
#need to resize image to same size as training images
#predict method expects a 2D array as input, so we convert our image
test_image = img_to_array(test_image) #gives a numpy array
#we have a dimension : batch as well, axis = 0 means that the dimension that we are adding will be the first dimension, we set batch as the first dimension
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image/255.0) # divide by 255 for normalization
#test_image is in the right format required
#1:dog, 0:cat - to find this we use:
training_set.class_indices
if result[0][0] > 0.5:   #[0][0] : first image in the first dimension(batch)
  prediction = 'dog'     #confidence level set : 0.5
else:
  prediction = 'cat'

1/1 [==============================] - 0s 39ms/step


In [34]:
print(prediction)

cat
